# Regresión logística

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from math import exp, log
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

# Ejemplo de regresión logística

In [ ]:
M = pd.read_csv("C:\\Users\\ricardo\\Desktop\\BanxicoTrabajoCasa_v3\\CursoCienciaDeDatos\\Clases\\RegresionLogistica\\EjemploRegresionLogistica3D.csv",dtype='str',encoding = "ISO-8859-1")
M[["Y","X1","X2","X3"]] = M[["Y","X1","X2","X3"]].astype(float)
M.head()

In [ ]:
#Regresión logística
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(M[["X1","X2","X3"]], M[["Y"]])
LR.predict(M[["X1","X2","X3"]])

In [ ]:
print(LR.coef_)
print(LR.intercept_)

In [ ]:
M["Y_aprox_1"] = LR.intercept_[0] + LR.coef_[0][0]*M["X1"] + LR.coef_[0][1]*M["X2"] + LR.coef_[0][2]*M["X3"]
M["Y_aprox_2"] = 1/(1+np.exp(-M["Y_aprox_1"] ))
M.head()

In [ ]:
def Etiqueta(x):
    if x.Y_aprox_2 >= 0.5:
        resp = 1
    else:
        resp = 0
    return resp
M["Y_aprox"] = M.apply(lambda x: Etiqueta(x),axis=1)
M.head()

In [ ]:
print(M["Y_aprox"].tolist())

In [ ]:
y_pred = LR.predict(M[["X1","X2","X3"]])
print(y_pred)

In [ ]:
np.array(M["Y"].tolist())

In [ ]:
confusion_matrix(M["Y_aprox"], y_pred)

In [ ]:
confusion_matrix(M["Y"], y_pred)

# Estimación de coeficientes mediante descenso por gradiente

In [ ]:
B = M[["X1","X2","X3"]].copy(deep=True).reset_index(drop=True)
B.head()

In [ ]:
B.iloc[1,:].tolist()
vx = [1]
vx.extend(B.iloc[1,:].tolist())
vx

In [ ]:
def PasoGradiente(w0,y,x,l):
    m = len(w0)
    n = len(y)
    vparcial = []
    for i in range(m):
        suma = 0
        for j in range(n):
            vx = [1]
            vx.extend(x.iloc[j,:].tolist())
            suma = suma + ( (1/(1+exp(-np.dot(w0,vx)))) - y[j] )*(vx[i])
        vparcial.extend([suma/n])
    w = []
    for i in range(m):
        w.extend([w0[i] - l*vparcial[i]])
    return w
#Ejemplo
y = M[["Y"]].copy(deep=True).reset_index(drop=True)
y = y["Y"].tolist()
x = M[["X1","X2","X3"]].copy(deep=True).reset_index(drop=True)
w0 = [-4,2,2,2] 
l = 0.01
w = PasoGradiente(w0,y,x,l)
print(w)    

In [ ]:
#Función de costo
def FCosto(w,y,x):
    n = len(y)
    suma = 0
    for j in range(n):
        vx = [1]
        vx.extend(x.iloc[j,:].tolist())
        suma = suma + y[j]*log(1/(1+exp(-np.dot(w,vx)))) + (1-y[j])*log(1 - 1/(1+exp(-np.dot(w,vx))))
    suma = (-1/n)*suma
    return suma
#Ejemplo
y = M[["Y"]].copy(deep=True).reset_index(drop=True)
y = y["Y"].tolist()
x = M[["X1","X2","X3"]].copy(deep=True).reset_index(drop=True)
w = [-4,3,3,3] 
C = FCosto(w,y,x)
print(C) 

In [ ]:
#Iteración de gradiente
N = 1000
y = M[["Y"]].copy(deep=True).reset_index(drop=True)
y = y["Y"].tolist()
x = M[["X1","X2","X3"]].copy(deep=True).reset_index(drop=True)
l = 0.01
w0 = [-4,3,3,3] 
vC = []
for i in range(N):
    if i % 100 == 0:
        print(i,"->",end="")
    w = PasoGradiente(w0,y,x,l)
    vC.extend([FCosto(w,y,x)])
    w0 = w
print()
print(w)

In [ ]:
#Gráfica función de costo
plt.title("Función de Costo")
plt.plot(range(len(vC)),vC,color="darkblue")
plt.xlabel('Iteración')
plt.show()

In [ ]:
#Comparativo base
B["Y_aprox_1"] = w[0] + w[1]*B["X1"] + w[2]*B["X2"] + w[3]*B["X3"]
B["Y_aprox_2"] = 1/(1+np.exp(-M["Y_aprox_1"] ))
B["Y_aprox"] = B.apply(lambda x: Etiqueta(x),axis=1)
B.head()

In [ ]:
print(B["Y_aprox"].tolist())

In [ ]:
LR.predict(B[["X1","X2","X3"]])

In [ ]:
np.array(M["Y"].tolist())

In [ ]:
confusion_matrix(LR.predict(B[["X1","X2","X3"]]),B["Y_aprox"])

In [ ]:
confusion_matrix(y,B["Y_aprox"])